In [78]:
#библиотека для регулярных выражений
import pandas as pd
import re 
ratings_movies = pd.read_csv('ratings_movies.csv')
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None
    

In [79]:
ratings_movies.tail()

,Unnamed: 0,userId,movieId,rating,date,title,genres
100831,100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller
100832,100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror
100834,100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi
100835,100835,610,170875,3.0,2017-05-03 21:20:15,The Fate of the Furious (2017),Action|Crime|Drama|Thriller


In [80]:
ratings_movies['date'] = ratings_movies['date'].astype('datetime64[ns]')

In [81]:
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)

In [82]:
ratings_movies['year_release'].isna().sum()

18

In [83]:
mask = (ratings_movies['year_release'] == 1999)
ratings_movies[mask].groupby(by='title')['rating'].mean().sort_values(ascending=True)


title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [84]:
mask2 = (ratings_movies['year_release'] == 2010)
ratings_movies[mask2].groupby(by='genres')['rating'].mean().sort_values()

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [85]:

ratings_movies.groupby('userId')['genres'].nunique().sort_values(ascending=False)

userId
599    524
414    482
448    403
380    399
474    395
      ... 
578     15
12      15
85      13
214     13
245     13
Name: genres, Length: 610, dtype: int64

In [86]:
ratings_movies.groupby('userId')['rating'].agg(
    ['count', 'mean']
    ).sort_values(by=['count','mean'], ascending=[True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [87]:
mask3 = (ratings_movies['year_release'] == 2018)
rate = ratings_movies[mask3].groupby('genres')['rating'].agg(
    ['mean', 'count']
).sort_values(by=['mean', 'count'], ascending=[False, False])
rate2 = (rate['count'] > 10)
rate[rate2]

,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [88]:
ratings_movies['year_rating'] = ratings_movies['date'].dt.year
pivot = ratings_movies.pivot_table(
    index = 'year_rating',
    columns='genres',
    values='rating',
    aggfunc='mean',
    fill_value=0)
pivot.columns
pivot['Animation|Children|Mystery']

year_rating
1996    0
1997    0
1998    0
1999    0
2000    0
2001    0
2002    0
2003    0
2004    0
2005    0
2006    0
2007    0
2008    0
2009    0
2010    0
2011    0
2012    0
2013    0
2014    0
2015    0
2016    0
2017    0
2018    5
Name: Animation|Children|Mystery, dtype: int64

In [89]:
orders = pd.read_csv('orders.csv', sep=';')


In [90]:
products = pd.read_csv('products.csv', sep=';')

In [91]:
orders

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3


In [92]:
products

,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR
5,86,"Носки Простые, муж",45,RUR
6,91,"Носки Честные, муж",50,RUR
7,103,"Носки Подарочные, муж",199,RUR
8,104,"Носки Подарочные, жен",249,RUR
9,124,Носки беговые Camino,999,RUR


In [93]:
order_products = orders.merge(
    products,
    left_on='ID товара',
    right_on='Product_ID'
)
order_products

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103,"Носки Подарочные, муж",199,RUR
1,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103,"Носки Подарочные, муж",199,RUR
2,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,103,"Носки Подарочные, муж",199,RUR
3,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,86,"Носки Простые, муж",45,RUR
4,08.11.2019 08:36:20,1,5,"Оплачен, формируется к отправке",Да,Нет,Нет,86,1,86,"Носки Простые, муж",45,RUR
5,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,104,"Носки Подарочные, жен",249,RUR
6,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,104,"Носки Подарочные, жен",249,RUR
7,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,104,"Носки Подарочные, жен",249,RUR
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,104,"Носки Подарочные, жен",249,RUR
9,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,124,Носки беговые Camino,999,RUR


In [94]:
mask5 = (order_products['Отменен'] == 'Да')
order_products[mask5]

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
9,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,124,Носки беговые Camino,999,RUR


In [100]:
order_products['profit'] = (order_products['Количество'] * order_products['Price'])
mask4 = order_products['Оплачен'] == 'Да'
order_products[mask4].groupby('ID Покупателя')['profit'].sum().sort_values(ascending=False)

ID Покупателя
7    17096
5    13043
8     1344
Name: profit, dtype: int64